In [20]:
from datetime import datetime
import numpy as np
import pandas as pd
from importer import Importer
from df_utils import group_weighted_mean_factory, add_new_entries, get_average_price_df, get_qtd, \
    get_exceptional_earnings_since_2018, get_last_state_from_2021

raw_df = Importer().process_files()
df = raw_df
df.data = raw_df.data.apply(pd.to_datetime)
df.qtd = df.apply(lambda line: line.qtd if line.compra_venda == "C" else -line.qtd, axis=1)

print(raw_df)

weighted_mean = group_weighted_mean_factory(raw_df, "qtd")

df = raw_df.sort_values("data").groupby(by= ["data", "abbreviation", "compra_venda"], axis=0) \
    .aggregate({"qtd": "sum", "preco": "mean",
                "valor_operacao": "sum", "taxas": "max", "total_ajustado": "sum", "preco_ajustado": weighted_mean})
df = df.reset_index()

# ------------- Extra entries -------------
# df = get_exceptional_earnings_since_2018(df)

# df = get_last_state_from_2021(df)


to_date = datetime(2022, 12, 31)
# date = datetime(2018, 4, 27)

average_price_df = get_average_price_df(df, to_date)
# display(average_price_df)
qtd_df = get_qtd(df, to_date)

final_df = average_price_df.merge(qtd_df, on="abbreviation")
final_df.columns = ["preco_medio", "qtd"]
final_df = final_df[final_df.qtd != 0]
print(final_df)

1 de 1 documentos processados.
        data compra_venda abbreviation      titulo ativo    qtd  preco  \
0 2020-10-11            C        AGRO3  BRASILAGRO    ON  100.0  23.21   
1 2020-10-11            C        AGRO3  BRASILAGRO    ON  300.0  23.21   
2 2020-10-11            C        AGRO3  BRASILAGRO    ON  100.0  23.21   

   valor_operacao  taxas  total_ajustado  preco_ajustado  
0          2321.0   6.46         2327.46       23.274600  
1          6963.0   6.46         6969.46       23.231533  
2          2321.0   6.46         2327.46       23.274600  
              preco_medio    qtd
abbreviation                    
AGRO3            23.24876  500.0


In [18]:
df[df.abbreviation == "ENAT3"].head(60)

,data,abbreviation,compra_venda,qtd,preco,valor_operacao,taxas,total_ajustado,preco_ajustado
